In [8]:
import torch
from torch import nn
import torch.nn.functional as F

import numpy as np

In [ ]:
class EspOptimizer(torch.optim.Optimizer):
    """

    """
    def __init__(self, params, lr):
        super().__init__(params, defaults={'lr': lr})

    def step(self):
        pass

In [2]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x):

        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)

        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [9]:
net = Net(n_input=2).to(device)

In [12]:
for parameter in net.parameters():
    print(parameter)
    break

Parameter containing:
tensor([[ 5.9152e-01, -2.9711e-01],
        [-5.1869e-01,  6.9431e-01],
        [ 4.1480e-01,  5.4345e-01],
        [-3.3186e-01, -4.4981e-01],
        [-3.1292e-02, -5.9555e-01],
        [-5.7582e-01,  2.6707e-01],
        [ 5.6602e-02,  6.4653e-01],
        [ 6.1133e-01, -3.3942e-01],
        [-9.8532e-02,  1.9221e-01],
        [-2.9972e-02, -6.9953e-01],
        [ 5.6790e-01,  2.1745e-01],
        [ 4.7219e-01,  2.8899e-01],
        [ 2.9043e-01, -6.8306e-01],
        [-6.8395e-01, -2.6371e-01],
        [-7.4724e-02, -4.3051e-01],
        [ 2.2411e-01, -3.6682e-01],
        [-1.7248e-01,  1.5230e-02],
        [ 5.4999e-01, -3.2879e-01],
        [ 7.8122e-02, -6.5292e-01],
        [-4.3029e-01,  2.0722e-01],
        [ 6.9959e-02,  4.2203e-01],
        [-6.9561e-01,  5.3183e-01],
        [-1.3878e-02,  2.7656e-01],
        [-5.9742e-01, -2.7688e-01],
        [-1.5536e-01, -4.1506e-01],
        [ 1.3531e-01,  4.7819e-01],
        [ 1.9502e-01,  3.7489e-01],
      

In [ ]:

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
loss = nn.BCELoss()
global_accuracy_train = []
global_accuracy_val = []
for i in range(150):
    accuracy_train = []
    train_gen = generate_batch(X_train, y_train, batch_size=256)
    val_gen = generate_batch(X_test, y_test, batch_size=256)
    for x, y in train_gen:
        x_tensor = torch.from_numpy(x).float().to(device)
        y_tensor = torch.from_numpy(y.values).double().to(device)
        optimizer.zero_grad()
        y_pred = net.forward(x_tensor)
        loss_value = loss(y_pred.view(-1).double(), y_tensor)
        accuracy_train.append(accuracy_score(y_pred.view(-1).long().detach().cpu().numpy(), y.values))
        loss_value.backward()
        optimizer.step()

    accuracy_val = []
    for x, y in val_gen:
        with torch.no_grad():
            x_tensor = torch.from_numpy(x).float().to(device)
            y_pred = net.forward(x_tensor).to(device)
            accuracy_val.append(accuracy_score(y_pred.view(-1).long().detach().cpu().numpy(), y))
    if i % 5 == 0:
        print(f'Эпоха {i}, Точность: На обучающей: {np.around(np.mean(accuracy_train)*100, 4)}%, на валидационной: {np.around(np.mean(accuracy_val)*100, 4)}%')
    global_accuracy_train.append(np.mean(accuracy_train))
    global_accuracy_val.append(np.mean(accuracy_val))

x_tensor = torch.from_numpy(X_test).float().to(device)
y_pred = net.forward(x_tensor).to(device)
acc = accuracy_score(y_pred.view(-1).long().detach().cpu().numpy(), y_test)
print(f'Точность на тестовой: {np.around(acc*100, 4)}%')